# Swarm Learning with Cross-Site Evaluation

In this example, we will demonstrate the Swarm Learning and Client-Controlled Cross-Site Evaluation Workflows using the Client API and the CIFAR10 dataset. 

## Swarm Learning

<img src="figs/swarm_learning.png" alt="swarm ccwf" width=35% height=35% />

Swarm Learning is a decentralized Federated Averaging algorithm where the key difference is that the server is not trusted with any sensitive information. The server is now only responsible for job health and lifecycle management via the `SwarmServerController`, while the clients are now responsible for training and aggregation logic via the swarm client-controlled `SwarmClientController`.

- `SwarmServerController`: manages swarm job lifecycle and configurations such as `aggr_clients` and `train_clients`
- `SwarmClientController`: sends `learn_task`  to all training clients to invoke their executors for `train` task each round, and sends results to designated `aggr_client` for aggregation.

Required tasks: `train`

See the full definitions of [SwarmServerController](https://github.com/NVIDIA/NVFlare/blob/main/nvflare/app_common/ccwf/swarm_server_ctl.py) and [SwarmClientController](https://github.com/NVIDIA/NVFlare/blob/main/nvflare/app_common/ccwf/swarm_client_ctl.py) for all available arguments.

## Client-Controlled Cross-Site Evaluation

<img src="figs/client_controlled_cse.png" alt="cse ccwf" width=35% height=35% />

In client-controlled cross-site evaluation, rather than sending client models to the server for distribution, clients instead communicate directly with each other to share their models for validation.

See the [cse example](../cse/cse.ipynb) for more details on server-controlled cross-site evaluation for a comparison.

- `CrossSiteEvalServerController`: manages evaluation workflow and configurations such as `evaluators` and `evaluatees`
- `CrossSiteEvalClientController`: sends `eval` request to evaluators, evaluators send `get_model` task to evaluatees, evaluatees send their model back with `submit_model`, and evaluators perform `validate` on the model and send the results to the server. 

Required tasks: `validate`, `submit_model`

See the full definitions of [CrossSiteEvalServerController](https://github.com/NVIDIA/NVFlare/blob/main/nvflare/app_common/ccwf/cse_server_ctl.py) and [CrossSiteEvalClientController](https://github.com/NVIDIA/NVFlare/blob/main/nvflare/app_common/ccwf/cse_client_ctl.py) for all available arguments.

## Converting DL training code to FL training code
We will be using the [Client API FL code](../code/fl/train.py) trainer converted from the original [Training a Classifer](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html) example.

For more details on using the Client API with multi-task support, see [Converting DL training code to FL training code with Multi-Task Support](../cse/cse.ipynb#code).



## Prepare Data

Make sure the CIFAR10 dataset is downloaded with the following script:

In [ ]:
! python ../data/download.py

## Job API

Let's use the Job API to create a CCWF Job.

We use the `add_swarm()` function to add our server_config, client_config, and cse_config.

First add the `SwarmServerConfig` with the number of rounds for the `SwarmServerController`.

We also add `CrossSiteEvalConfig` for the `CrossSiteEvalServerController`.

On the client side, we add the `SwarmClientConfig` for the `SwarmClientController` which maps the `learn_task_name` to `train` and add the `CrossSiteEvalClientController` which uses the `validate` and `submit_model` tasks. These task are handled by the `ScriptRunner` with our `train.py` script. Additionally, required components including the persistor with the initial `Net()` model, aggregator, and shareable generator are defined as client-side components.

Let's first copy the required files:

In [ ]:
! cp ../code/fl/train.py train.py
! cp ../code/fl/net.py net.py

Then we can use Job API to easily create a job and run in simulator:

In [ ]:
from net import Net

from nvflare.apis.dxo import DataKind
from nvflare.app_common.aggregators.intime_accumulate_model_aggregator import InTimeAccumulateWeightedAggregator
from nvflare.app_common.ccwf.ccwf_job import CCWFJob, CrossSiteEvalConfig, SwarmClientConfig, SwarmServerConfig
from nvflare.app_common.ccwf.comps.simple_model_shareable_generator import SimpleModelShareableGenerator
from nvflare.app_opt.pt.file_model_persistor import PTFileModelPersistor
from nvflare.job_config.script_runner import ScriptRunner

n_clients = 2
num_rounds = 3
train_script = "train.py"

job = CCWFJob(name="swarm")
aggregator = InTimeAccumulateWeightedAggregator(expected_data_kind=DataKind.WEIGHTS)
job.add_swarm(
    server_config=SwarmServerConfig(num_rounds=num_rounds),
    client_config=SwarmClientConfig(
        executor=ScriptRunner(script=train_script),
        aggregator=aggregator,
        persistor=PTFileModelPersistor(model=Net()),
        shareable_generator=SimpleModelShareableGenerator(),
    ),
    cse_config=CrossSiteEvalConfig(eval_task_timeout=300),
)

job.export_job("/tmp/nvflare/jobs/job_config")
job.simulator_run("/tmp/nvflare/jobs/workdir", n_clients=n_clients, gpu="0")

## Run the Job

The previous cell exports the job config and executes the job in NVFlare simulator.

If you want to run in production system, you will need to submit this exported job folder to nvflare system.

As an additional resource, also see the [Swarm Learning Example](../../../../advanced/swarm_learning/README.md) which utilizes the CIFAR10 ModelLearner instead of the Client API.

Congratulations! You have completed the CIFAR10 step-by-step example series.
Next take a look at the [higgs](../../higgs/README.md) example series for how to use machine learning methods for federated learning on tabular data.